In [ ]:
'''
model cell
'''
# import the necessary libraries
import warnings
warnings.filterwarnings("ignore")

import keras
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution1D
from tensorflow.keras.layers import Dropout, Input, BatchNormalization, MaxPooling1D, concatenate
from tensorflow.keras.models import Model
#from keras.utils import multi_gpu_model
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn import preprocessing
import joblib


import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import time

def import_tensorflow():
    # Filter tensorflow version warnings
    import os
    # https://stackoverflow.com/questions/40426502/is-there-a-way-to-suppress-the-messages-tensorflow-prints/40426709
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
    import warnings
    # https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning
    warnings.simplefilter(action='ignore', category=FutureWarning)
    warnings.simplefilter(action='ignore', category=Warning)
    import tensorflow as tf
    tf.get_logger().setLevel('INFO')
    tf.autograph.set_verbosity(0)
    import logging
    tf.get_logger().setLevel(logging.ERROR)
    return tf

import os

tf = import_tensorflow()

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="4,5" 


In [ ]:
merge_all = pd.read_csv("/data/dp2/data/justin/data/merge_all_norm_resp_20191203-20.csv")
path_indexer_df = pd.read_csv("/data/dp2/data/justin/reall/file_path_index_2023_Norm.csv")

SigV500_df = pd.read_pickle('/data/dp2/data/justin/reall/2023_feature+SigV500.pkl')


# Plot_CEG_func

In [ ]:
def plot_test (Y_vali1, vali_pred1,name,plot=True):
    fig = plt.figure(figsize=(11,7),dpi=100)
    
    
    ax1 = fig.add_axes([0, 0.25, 1, 0.65])

    ax3 = fig.add_axes([0.15, 0.05, 0.7, 0.15])

    lower_bound=70
    upper_bound=300
    #fig.suptitle('Horizontally stacked subplots')

    #ax1.set_title("Test",fontsize=15)
    ax1.scatter(Y_vali1,vali_pred1 ,facecolors='none',edgecolors='blue',alpha=0.5)
    ax1=plot_CEG(ax1)
    scores ,scores_lst= cal_CEG_score(Y_vali1,vali_pred1 )
    the_table =ax3.table(cellText=scores,colLabels=['A','B','C','D','E'],loc='center'
                         ,cellLoc='center',fontsize=13.5,bbox=(0.15, 0, 0.7, 0.7))
    the_table.scale(1,1.6)
    ax3.axis("off")

    fig.suptitle("CEG Analysis of {}".format(name),fontsize=18)
    
    #plt.savefig('Model_Performance_pic/test_{}_{}.png'.format(name,iteration))
    if plot:
        plt.show()
    else:
        plt.close()
        
def plot_CEG(ax):
    
    ax.set_xlabel("Finger Prick Measured Concentration [mg/dl]",fontsize=13.5)
    ax.set_ylabel("Predicted Concentration [mg/dl]",fontsize=13.5)
    ax.set_xticks([0, 50, 100, 150, 200, 250, 300, 350, 400])
    ax.set_yticks([0, 50, 100, 150, 200, 250, 300, 350, 400])
    ax.set_facecolor('white')

    #Set axes lengths
    ax.set_xlim([0, 400])
    ax.set_ylim([0, 400])
    ax.set_aspect((400)/(400))

    #Plot zone lines
    ax.plot([0,400], [0,400], ':', c='black')                      #Theoretical 45 regression line
    ax.plot([0, 175/3], [70, 70], '-', c='black')
    #plt.plot([175/3, 320], [70, 400], '-', c='black')
    ax.plot([175/3, 400/1.2], [70, 400], '-', c='black')           #Replace 320 with 400/1.2 because 100*(400 - 400/1.2)/(400/1.2) =  20% error
    ax.plot([70, 70], [84, 400],'-', c='black')
    ax.plot([0, 70], [180, 180], '-', c='black')
    ax.plot([70, 290],[180, 400],'-', c='black')
    # plt.plot([70, 70], [0, 175/3], '-', c='black')
    ax.plot([70, 70], [0, 56], '-', c='black')                     #Replace 175.3 with 56 because 100*abs(56-70)/70) = 20% error
    # plt.plot([70, 400],[175/3, 320],'-', c='black')
    ax.plot([70, 400], [56, 320],'-', c='black')
    ax.plot([180, 180], [0, 70], '-', c='black')
    ax.plot([180, 400], [70, 70], '-', c='black')
    ax.plot([240, 240], [70, 180],'-', c='black')
    ax.plot([240, 400], [180, 180], '-', c='black')
    ax.plot([130, 180], [0, 70], '-', c='black')

    #Add zone titles
    ax.text(30, 15, "A", fontsize=15)
    #ax.text(370, 260, "B", fontsize=15)
    #ax.text(280, 370, "B", fontsize=15)
    #ax.text(160, 370, "C", fontsize=15)
    ax.text(160, 15, "C", fontsize=15)
    ax.text(30, 140, "D", fontsize=15)
    #ax.text(370, 120, "D", fontsize=15)
    #ax.text(30, 370, "E", fontsize=15)
    #ax.text(370, 15, "E", fontsize=15)
    
    return ax

def cal_CEG_score (ref_values1,pred_values1):
    zone1 = [0] * 5
    for i in range(len(ref_values1)):
        if (ref_values1[i] <= 70 and pred_values1[i] <= 70) or (pred_values1[i] <= 1.2*ref_values1[i] and pred_values1[i] >= 0.8*ref_values1[i]):
            zone1[0] += 1    #Zone A
 
        elif (ref_values1[i] >= 180 and pred_values1[i] <= 70) or (ref_values1[i] <= 70 and pred_values1[i] >= 180):
            zone1[4] += 1    #Zone E
 
        elif ((ref_values1[i] >= 70 and ref_values1[i] <= 290) and pred_values1[i] >= ref_values1[i] + 110) or ((ref_values1[i] >= 130 and ref_values1[i] <= 180) and (pred_values1[i] <= (7/5)*ref_values1[i] - 182)):
            zone1[2] += 1    #Zone C
        elif (ref_values1[i] >= 240 and (pred_values1[i] >= 70 and pred_values1[i] <= 180)) or (ref_values1[i] <= 175/3 and pred_values1[i] <= 180 and pred_values1[i] >= 70) or ((ref_values1[i] >= 175/3 and ref_values1[i] <= 70) and pred_values1[i] >= (6/5)*ref_values1[i]):
            zone1[3] += 1    #Zone D
        else:
            zone1[1] += 1    #Zone B
    
    z1=zone1
    z1_all = z1[0]+z1[1]+z1[2]+z1[3]+z1[4]
    z1_a = round((z1[0]/z1_all)*100,1)
    z1_b = round((z1[1]/z1_all)*100,1)
    z1_c = round((z1[2]/z1_all)*100,1)
    z1_d = round((z1[3]/z1_all)*100,1)
    z1_e = round((z1[4]/z1_all)*100,1)
    data=[[str(z1_a)+'%',str(z1_b)+'%',str(z1_c)+'%',str(z1_d)+'%',str(z1_e)+'%']]    
    
    return data,[z1_a , z1_b , z1_c , z1_d , z1_e]

# Declare-Model

In [ ]:

def Deduction_Learning(include_top=True, h_activation = 'relu', output_activation = 'linear', learn_rate = 0.00001, 
                       beta2 = 0.95, loss_param = 'mean_squared_error'):
    '''
    include_top: whether to include the 3 fully-connected layers at the top of the network.
    input_shape : the deep learning inputs from the input PPG signal
    output_activation : the last regressin layer activation
    loss_param :our argument to compile the training model
    loss_param = 'mean_squared_error'
    '''
    input_shape=Input(shape=(413, 1),name= "net_input")
    # Block 1
    x = Convolution1D(filters=256/2, kernel_size=10, name="block1_conv1")(input_shape)
    x = BatchNormalization(axis=-1, name="block1_batchnorm")(x)
    x = Activation(h_activation, name="block1_act")(x)
    x = MaxPooling1D(pool_size=2, name="block1_pool")(x)
    # Block 2
    x = Convolution1D(filters=256/2, kernel_size=10, name="block2_conv1") (x)
    x = BatchNormalization(axis=-1, name="block2_batchnorm")(x)
    x = Activation(h_activation, name="block2_act")(x)
    x = MaxPooling1D(pool_size=2, name="block2_pool")(x)
    # Block 3
    x = Convolution1D(filters=512/2, kernel_size=10, name="block3_conv1") (x)
    x = BatchNormalization(axis=-1, name="block3_batchnorm")(x)
    x = Activation(h_activation, name="block3_act")(x)
    x = MaxPooling1D(pool_size=2, name="block3_pool")(x)

    # regression block
    if include_top:
        x = Flatten(name="flatten")(x)
        #input1 = keras.layers.Input(shape=(1,))
        x = BatchNormalization(axis=-1, name = "batch_norm")(x)
        #x = keras.layers.Concatenate(axis=-1)([x, input1])
        x = Dense(2048/2,activation=h_activation, name="FC1")(x)
        x = Dense(1024/2,activation=h_activation, name="FC2")(x)
        x = Dense(1,activation=output_activation, name="predictions")(x)
    # Create model.
    model = Model(inputs=[input_shape], outputs=[x])
    
    Optimizer=Adam(lr=learn_rate, beta_1=0.9, beta_2=beta2)
    model.compile(loss=loss_param, optimizer=Optimizer)
    return model

In [ ]:
#data_df=pd.read_pickle('2023_DL_SinglePretest_Set1_noDM_Drug.pkl')
data_df =SigV500_df

BL_drug_cond=(merge_all['BL_drug']==0)
BP_drug_cond=(merge_all['BP_drug']==0)
DM_drug_cond=(merge_all['DM_drug']==0)
DM_inject_cond=(merge_all['DM_inject']==0)
O_drug_cond=(merge_all['O_drug']==0)

noDM_drug_id = merge_all.loc[(merge_all['Time']==0)&(DM_drug_cond)&DM_inject_cond]['person_time_idx']  #不含糖尿藥物

DM_drug_id = merge_all.loc[(merge_all['Time']==0)&
                           BL_drug_cond&BP_drug_cond&
                           (~DM_drug_cond)&DM_inject_cond&
                           O_drug_cond]['person_time_idx']  #不含胰島素

anyDM_drug_id=merge_all.loc[(merge_all['Time']==0)&(~BL_drug_cond|~BP_drug_cond|~O_drug_cond)&
                           (~DM_drug_cond|~DM_inject_cond)]['person_time_idx'] # 有服用血糖藥 或 有胰島素注射 同時混用其他藥物



set1_target_Base_pids = list( merge_all.loc[(merge_all['count_time']>=2)&(merge_all['HbA1C']>6.5)&(merge_all['person_time_idx'].isin(noDM_drug_id))]['count'].unique())
print(set1_target_Base_pids )

set2_target_Base_pids = list( merge_all.loc[(merge_all['count_time']>=3)&(merge_all['person_time_idx'].isin(DM_drug_id))]['count'].unique())


set3_target_Base_pids = list( merge_all.loc[(merge_all['count_time']>=3)&(merge_all['person_time_idx'].isin(anyDM_drug_id))]['count'].unique())

print("SET 1 符合條件受測者共 {} 人".format(len(set1_target_Base_pids)))
print(set1_target_Base_pids )

In [ ]:
f1=['HR','Age','FW_50','Total_Area',
     'LFP_valley','HFP_valley','pNN20_valley',
    'SDNN_valley','TP_valley','UT','Area50',
    'Total_Area/HR','area']
data_df[f1] 

In [ ]:
set1_target_Base_pids=[897.0, 1009.0, 245.0, 342.0, 1304.0]

train_set=[ 235.0, 897.0, 1009.0, 245.0, 342.0, 1768.0, 1628.0, 920.0, 1304.0, 789.0, 1486.0, 1725.0, 1726.0, 1592.0, 1891.0, 745.0]


f1=['HR','Age','FW_50','W_cir',
     'LFP_valley','HFP_valley','pNN20_valley',
    'SDNN_valley','TP_valley','UT','Area50',
    'Total_Area/HR','area']

#f1_lst=[f+"_lst" for f in f1 ]

f2=['last_BG']


feature_scaler = preprocessing.MinMaxScaler().fit(data_df[f1])

for target_pid in set1_target_Base_pids:
    
    print('Test Target {}'.format(target_pid))
    test_pid_t_idx = merge_all.loc[(merge_all['Person No'].isin([target_pid]))|(merge_all['count'].isin([target_pid]))]['person_time_idx'] #get test set pid_t_idx
    group = merge_all.loc[((merge_all['Person No'].isin(train_set))|(merge_all['count'].isin(train_set)))&(merge_all['HbA1C']<7.5)]['person_time_idx'] ## Use >=7.5 for Gated(H) <7.5 for Gated(L)
    
    train_pid_t_idx = data_df.loc[(data_df['person_time_idx'].isin(group))&(~data_df['person_time_idx'].isin(test_pid_t_idx))]['person_time_idx']
    
    X_train_s = data_df.loc[data_df['person_time_idx'].isin(train_pid_t_idx)].iloc[:,21:421]
    X_test_s  = data_df.loc[data_df['person_time_idx'].isin(test_pid_t_idx)].iloc[:,21:421]

    sample_weight = data_df.loc[(data_df['person_time_idx'].isin(group))&(~data_df['person_time_idx'].isin(test_pid_t_idx))]['BG'].astype(int)
    _sample_weight=[]
    for w in sample_weight:
        if (w <=120) & (w >=80) :
            _sample_weight.append( 1)
        else:
             _sample_weight.append( 1 )

    _sample_weight=pd.Series(_sample_weight)

    

    t = data_df['BG']
    t = t.values.reshape(-1, 1)
    scaler_BG = preprocessing.MinMaxScaler().fit(t)

    t1 = data_df[f1] 

    scaler_feat = preprocessing.MinMaxScaler().fit(t1)


    Y_train = data_df.loc[data_df['person_time_idx'].isin(train_pid_t_idx)][['BG']]
    Y_train = scaler_BG.transform(Y_train)

    Y_test = data_df.loc[data_df['person_time_idx'].isin(test_pid_t_idx)][['BG']]
    Y_test = scaler_BG.transform(Y_test)

    # reshape train data
    X_train_r_feat = data_df.loc[data_df['person_time_idx'].isin(train_pid_t_idx)][f1]
    X_train_r_feat = scaler_feat.transform(X_train_r_feat)


    # reshape vali data
    X_test_r_feat = data_df.loc[data_df['person_time_idx'].isin(test_pid_t_idx)][f1]
    X_test_r_feat = scaler_feat.transform(X_test_r_feat)


    #combine normalized features with signal
    X_train_s=X_train_s.reset_index(drop=True)
    X_train_s=X_train_s.merge(pd.DataFrame(X_train_r_feat,columns=f1),left_index=True, right_index=True)

    X_test_s=X_test_s.reset_index(drop=True)
    X_test_s=X_test_s.merge(pd.DataFrame(X_test_r_feat,columns=f1),left_index=True, right_index=True)


    # reshape train data
    X_train_r_s = np.zeros((len(X_train_s), 413, 1))
    X_train_r_s[:, :, 0] = X_train_s



    # reshape vali data
    X_test_r_s = np.zeros((len(X_test_s), 413, 1))
    X_test_r_s[:, :, 0] = X_test_s


    start = time.time() # Mark start trining time

    keras.backend.clear_session()

    #fit
    callback = EarlyStopping(monitor='val_loss', patience=20,restore_best_weights=True)

    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        model= Deduction_Learning()
    
    print("Train: ",X_train_r_s.shape,Y_train.shape,"Test: ",X_test_s.shape,Y_test.shape)
    history = model.fit([X_train_r_s], Y_train, epochs=250,
                        batch_size=500,validation_data=([X_test_r_s], Y_test),
                        verbose=0,sample_weight=_sample_weight) #, callbacks=[callback]

    end = time.time() # Mark end trining time
    # 輸出結果
    print("執行時間：%f 分鐘" % ((int(end) - int(start))/60))


    plt.figure(figsize=(6,2)) #plot training & testing loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylim(0,0.15)
    plt.grid(True)
    plt.show()

    train_pred  = model.predict([X_train_r_s])
    train_pred1 = scaler_BG.inverse_transform(train_pred)
    train_pred1 =  train_pred1.reshape(-1)

    Y_train = data_df.loc[data_df['person_time_idx'].isin(train_pid_t_idx)][['person_time_idx','BG']]
    Y_train['pred_BG'] = train_pred1

    train_result_df = pd.DataFrame()
    ref_BG=[]
    pred_BG=[]
    pid_t_idx =[]
    for pid in Y_train['person_time_idx'].unique():
        pid_t_idx.append(pid)
        ref_BG.append(Y_train.loc[Y_train['person_time_idx']==pid]['BG'].median()) 
        pred_BG.append(Y_train.loc[Y_train['person_time_idx']==pid]['pred_BG'].median()) 

    train_result_df['person_time_idx']=pid_t_idx
    train_result_df['pred_BG']=pred_BG
    train_result_df['ref_BG']=ref_BG

    plot_test(list(Y_train['BG']),list(Y_train['pred_BG']),'Training result')


    #build testing result table
    test_pred = model.predict([X_test_r_s])
    test_pred1 = scaler_BG.inverse_transform(test_pred)
    test_pred1=test_pred1.reshape(-1)
    ref_BG

    Y_test = data_df.loc[data_df['person_time_idx'].isin(test_pid_t_idx)][['person_time_idx','BG']]
    Y_test['pred_BG']=test_pred1

    ref_BG=[]
    pred_BG=[]
    std=[]
    ptile_25=[]
    ptile_75=[]
    passed=[]
    for pid_t_idx in test_pid_t_idx:
        try:
            ref_BG.append(int(data_df.loc[data_df['person_time_idx']==pid_t_idx]['BG'].values[0])) 
            pred_BG.append(Y_test.loc[Y_test['person_time_idx']==pid_t_idx]['pred_BG'].median()) 
            std.append(Y_test.loc[Y_test['person_time_idx']==pid_t_idx]['pred_BG'].std()) 
            ptile_25.append(Y_test.loc[Y_test['person_time_idx']==pid_t_idx]['pred_BG'].describe()['25%']) 
            ptile_75.append(Y_test.loc[Y_test['person_time_idx']==pid_t_idx]['pred_BG'].describe()['75%']) 
        except:
            passed.append(pid_t_idx)

    _test_pid_t_idx=test_pid_t_idx.loc[~test_pid_t_idx.isin(passed)]
    test_result_df = pd.DataFrame({'test_pid_t_idx':_test_pid_t_idx, 'ref_BG':ref_BG, 'pred_BG':pred_BG , 'std':std, "ptile_25":ptile_25, "ptile_75":ptile_75})  

    test_result_df.to_pickle("./result_tables/2024 NoPretest_Set1_downsized_gatedL- {}".format(target_pid)) # Save test result table

    plot_test(list(test_result_df['ref_BG']),list(test_result_df['pred_BG']),'Testing set {}'.format(target_pid))




In [ ]:
result_df=pd.DataFrame()
for pid in set1_target_Base_pids:
    temp_df = pd.read_pickle("./result_tables/2024 NoPretest_Set1_downsized- {}".format(pid))
    result_df = pd.concat([result_df,temp_df])


In [ ]:
plot_test(list(result_df['ref_BG']),list(result_df['pred_BG']),'Training result')



In [ ]:
 
plt.figure(figsize=(6,2)) #plot training & testing loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylim(0,0.1)

plt.grid()
plt.show()